In [1]:
# paramter cell do not remove!!
# nb_parm='datalake|raw/pdf|Birddiversityanddistribution|pdf||300|150'
# nb_parm='datalake|raw/text-csv|PFW_spp_translation_table_May2024|csv||300|150'
nb_parm='llmnok'
question = 'how many bird species are in migratory?'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
# collection = "Bridknowledge"

In [2]:
import sys
import os, io

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

## Do the task after this

In [3]:
import os
import boto3
from pathlib import Path
import subprocess
from botocore.config import Config

# === Configuration ===
minio_endpoint = os.getenv("MINIO_ENDPOINT", "http://localhost:9001/browser")
minio_access_key = os.getenv("MINIO_ACCESS_KEY", "minioadmin")
minio_secret_key = os.getenv("MINIO_SECRET_KEY", "minioadmin")
minio_bucket = "datalake"
minio_key = "Agent/packages/adk_short_bot-0.1.0-py3-none-any.whl"  # or .tar.gz if needed
# minio_key = "Agent/packages/adk_short_bot-0.1.0.tar.gz"
local_pkg_path = Path("/tmp/downloads") / Path(minio_key).name

# === Ensure download directory exists ===
local_pkg_path.parent.mkdir(parents=True, exist_ok=True)

# === Download from MinIO ===
s3 = boto3.client(
    's3',
    endpoint_url='http://minio:9000',  # Replace with your MinIO server URL
    aws_access_key_id='minioadmin',    # Use MinIO's access key
    aws_secret_access_key='minioadmin',# Use MinIO's secret key
    config=Config(signature_version='s3v4')
)

print(f"⬇️  Downloading {minio_key} from MinIO...")
s3.download_file(minio_bucket, minio_key, str(local_pkg_path))
print(f"✅ Downloaded to: {local_pkg_path}")

# === Uninstall package if found ===
package_name = "adk_short_bot"
print(f"⛔ Uninstalling any existing versions of {package_name}...")
subprocess.run(["pip", "uninstall", package_name, "-y"], check=False)

# === Install package ===
print(f"📦 Installing {local_pkg_path.name}...")
subprocess.run(["pip", "install", str(local_pkg_path)], check=True)
print("✅ Package installed.")


⬇️  Downloading Agent/packages/adk_short_bot-0.1.0-py3-none-any.whl from MinIO...
✅ Downloaded to: /tmp/downloads/adk_short_bot-0.1.0-py3-none-any.whl
⛔ Uninstalling any existing versions of adk_short_bot...


📦 Installing adk_short_bot-0.1.0-py3-none-any.whl...
Processing /tmp/downloads/adk_short_bot-0.1.0-py3-none-any.whl
✅ Package installed.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install "google-cloud-aiplatform[adk,agent_engines]"


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
# === Use your agent ===




import uuid

from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from adk_short_bot.agent import root_agent

load_dotenv()

# Create a new session service to store state
session_service_stateful = InMemorySessionService()

# Create a NEW session
APP_NAME = "Brandon Bot"
USER_ID = "brandon_hancock"
SESSION_ID = str(uuid.uuid4())
stateful_session = session_service_stateful.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)
print("CREATED NEW SESSION:")
print(f"\tSession ID: {SESSION_ID}")

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service_stateful,
)

new_message = types.Content(
    role="user", parts=[types.Part(text="Hey how are you today?")]
)

for event in runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=new_message,
):
    if event.is_final_response():
        if event.content and event.content.parts:
            print(f"Final Response: {event.content.parts[0].text}")




CREATED NEW SESSION:
	Session ID: a970e62e-526f-413f-a75f-01bb9c887c9d
Final Response: Original Character Count: 20
New Character Count: 14
New message: Hey, how are you?

